# 실습 문제 : ConvLab 프레임워크를 이용한 대화시스템 모델 평가 및 모델 선택

아래 코드에서는 이 실습문제에서 필요한 패키지를 import하고 helper function을 정의합니다.

In [1]:
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.dialog_agent import BiSession, PipelineAgent
from convlab2.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch
import spacy

import logging 
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
cntp_logger = logging.getLogger('urllib3.connectionpool')
cntp_logger.setLevel(logging.WARNING)
ttu_logger = logging.getLogger('transformers.tokenization_utils')
ttu_logger.setLevel(logging.WARNING)
tcu_logger = logging.getLogger('transformers.configuration_utils')
tcu_logger.setLevel(logging.WARNING)
tmu_logger = logging.getLogger('transformers.modeling_utils')
tmu_logger.setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.INFO)
import warnings
warnings.filterwarnings("ignore")

spacy.cli.download('en_core_web_sm')

def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)


/home/hanjy/anaconda3/envs/0813_dialogue_system_v5/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hanjy/anaconda3/envs/0813_dialogue_system_v5/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hanjy/anaconda3/envs/0813_dialogue_system_v5/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')



## ConvLab 에서 지원하는 모델들 load

이용가능한 model들:

- NLU: BERTNLU, MILU, SVMNLU
- DST: RuleDST
- Word-DST: SUMBT, TRADE (set `sys_nlu` to `None`)
- Policy: RulePolicy, Imitation, REINFORCE, PPO, GDPL
- Word-Policy: MDRG, HDSA, LaRL (set `sys_nlg` to `None`)
- NLG: Template, SCLSTM
- End2End: Sequicity, DAMD, RNN_rollout (directly used as `sys_agent`)
- Simulator policy: Agenda, VHUS (for `user_policy`)


아래 코드는 실습시간 때 사용했던 Step 3 코드들의 일부입니다.

In [2]:
# available NLU models
from convlab2.nlu.svm.multiwoz import SVMNLU
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
# available DST models
from convlab2.dst.rule.multiwoz import RuleDST
#from convlab2.dst.mdbt.multiwoz import MDBT
from convlab2.dst.sumbt.multiwoz import SUMBT
from convlab2.dst.trade.multiwoz import TRADE
# available Policy models
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.policy.ppo.multiwoz import PPOPolicy
from convlab2.policy.pg.multiwoz import PGPolicy
from convlab2.policy.mle.multiwoz import MLEPolicy
from convlab2.policy.gdpl.multiwoz import GDPLPolicy
from convlab2.policy.mdrg.multiwoz import MDRGWordPolicy
from convlab2.policy.hdsa.multiwoz import HDSA
from convlab2.policy.larl.multiwoz import LaRL
# available NLG models
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
from convlab2.e2e.sequicity.multiwoz import Sequicity
from convlab2.e2e.damd.multiwoz import Damd
spacy.load('en_core_web_sm')

Load from https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip


08/11/2020 19:32:50 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip not found in cache, downloading to /tmp/tmpaxbbg487
100%|██████████| 21577107/21577107 [00:21<00:00, 1024368.30B/s]
08/11/2020 19:33:12 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpaxbbg487 to cache at /home/hanjy/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/11/2020 19:33:12 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/hanjy/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/11/2020 19:33:12 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpaxbbg487


Extracting...
Load from https://convlab.blob.core.windows.net/convlab-2/mdrg_data.zip


08/11/2020 19:33:13 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/mdrg_data.zip not found in cache, downloading to /tmp/tmpeaog3ard
100%|██████████| 47104409/47104409 [00:30<00:00, 1547625.65B/s]
08/11/2020 19:33:44 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpeaog3ard to cache at /home/hanjy/.convlab2/cache/00a406587d87174b74198f14cae25cd2054923a471c59233f27ec80caef23686.da1518d0f3a98f95e2be9aee8474275aa3e182c5b9faccf16e9deac38752afce
08/11/2020 19:33:44 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/hanjy/.convlab2/cache/00a406587d87174b74198f14cae25cd2054923a471c59233f27ec80caef23686.da1518d0f3a98f95e2be9aee8474275aa3e182c5b9faccf16e9deac38752afce
08/11/2020 19:33:44 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpeaog3ard


Extracting...
Load from https://convlab.blob.core.windows.net/convlab-2/mdrg_db.zip


08/11/2020 19:33:48 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/mdrg_db.zip not found in cache, downloading to /tmp/tmpotpzv3zh
100%|██████████| 183081/183081 [00:00<00:00, 463036.19B/s]
08/11/2020 19:33:49 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpotpzv3zh to cache at /home/hanjy/.convlab2/cache/a9766cc757fb79e7ac5266715dd065c687f884e5dc06840bba4d3b07307eb95b.b7bac7303e20c54957b367fa386215aaa595d5df9fb04341554b2067d458679c
08/11/2020 19:33:49 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/hanjy/.convlab2/cache/a9766cc757fb79e7ac5266715dd065c687f884e5dc06840bba4d3b07307eb95b.b7bac7303e20c54957b367fa386215aaa595d5df9fb04341554b2067d458679c
08/11/2020 19:33:49 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpotpzv3zh


Extracting...


[nltk_data] Downloading package stopwords to /home/hanjy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.


Word-DST 모델들은 NLU와 DST가 합쳐진 모델을 의미합니다. 따라서, 별도의 NLU 모델 없이 사용할 수 있습니다.

따라서, (1) NLU+RuleDST 또는 (2) Word-DST 로 조합이 가능합니다.

[주의!] Word-DST 의 경우 sys_nlu = None 이어야 합니다.

Word-Policy 모델들은 Dialogue Policy 와 NLG 가 합쳐진 모델을 의미합니다. 따라서 별도의 NLG 모델없이 사용할 수 있습니다.

따라서, (1) Policy+NLG 또는 Word-Policy 로 조합이 가능합니다.

[주의!] Word-Policy 의 경우 sys_nlg = None 이어야 합니다.


아래 코드는 system module들의 예시입니다.

In [4]:
# NLU+RuleDST:
sys_nlu = MILU()
# sys_nlu = SVMNLU()
# sys_nlu = BERTNLU()
sys_dst = RuleDST()

# or Word-DST:
# sys_nlu = None
# sys_dst = SUMBT()
# sys_dst = TRADE()
#### (not working!) sys_dst = MDBT()

# [Caution] In Word-DST case, sys_nlu must be "None"

# Policy+NLG:
sys_policy = RulePolicy()
# sys_policy = PPOPolicy()
# sys_policy = PGPolicy()
# sys_policy = MLEPolicy()
# sys_policy = GDPLPolicy()
sys_nlg = TemplateNLG(is_user=False)
#sys_nlg = SCLSTM(is_user=False)

# or Word-Policy:
# sys_policy = LaRL()
# sys_policy = HDSA()
# sys_policy = MDRGWordPolicy()
# sys_nlg = None

# [Caution] "In Word-policy case, sys_nlg must be None"

sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')

## End-to-End model
# sys_agent = Sequicity()
# sys_agent = Damd()


Load from https://convlab.blob.core.windows.net/convlab-2/milu_multiwoz_all_context.tar.gz


08/11/2020 19:40:05 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/milu_multiwoz_all_context.tar.gz not found in cache, downloading to /tmp/tmpludctzgr
100%|██████████| 10943993/10943993 [00:05<00:00, 2095005.21B/s]
08/11/2020 19:40:11 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpludctzgr to cache at /home/hanjy/.convlab2/cache/adeae6e5151198de3b1634fcb630a0243cd2e098a75703260c4489c6e92a5f51.4fff038b1c36dbb489d93575f604cf72276b651ac6b370b5a48f8f67df2ed971
08/11/2020 19:40:11 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/hanjy/.convlab2/cache/adeae6e5151198de3b1634fcb630a0243cd2e098a75703260c4489c6e92a5f51.4fff038b1c36dbb489d93575f604cf72276b651ac6b370b5a48f8f67df2ed971
08/11/2020 19:40:11 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpludctzgr
08/11/2020 19:40:11 - INFO - allennlp.models.archival -   loading archive file /home/hanjy/.convlab2/cache/adeae6e51511

08/11/2020 19:40:11 - INFO - allennlp.common.params -   model.text_field_embedder.token_embedders.token_characters.encoder.conv_layer_activation = relu
08/11/2020 19:40:11 - INFO - allennlp.common.registrable -   instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
08/11/2020 19:40:11 - INFO - allennlp.common.params -   model.text_field_embedder.token_embedders.token_characters.encoder.output_dim = None
08/11/2020 19:40:11 - INFO - allennlp.common.params -   model.text_field_embedder.token_embedders.token_characters.dropout = 0.0
08/11/2020 19:40:11 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.modules.token_embedders.token_embedder.TokenEmbedder'> from params {'embedding_dim': 50, 'trainable': True, 'type': 'embedding'} and extras {'vocab'}
08/11/2020 19:40:11 - INFO - allennlp.common.params -   model.text_field_embedder.token_embedders.tokens.type = embedding
08/11/2020 19:40:11 - INFO - allennlp.common.params -   mode

08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.bias_hh_l0_reverse
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.bias_ih_l0
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.bias_ih_l0_reverse
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.weight_hh_l0
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.weight_hh_l0_reverse
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.weight_ih_l0
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      encoder._module.weight_ih_l0_reverse
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      intent_encoder._module.bias_hh_l0
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      intent_encoder._module.bias_hh_l0_reverse
08/11/2020 19:40:11 - INFO - allennlp.nn.initializers -      intent_encoder._module.bias_ih_l0
08/11/2020 19:40:11 - INFO - allennlp.nn.initialize

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/hanjy/anaconda3/envs/0813_dialogue_system_v5/lib/python3.7/site-packages/en_core_web_sm
-->
/home/hanjy/anaconda3/envs/0813_dialogue_system_v5/lib/python3.7/site-packages/spacy/data/en_core_web_sm
You can now load the model via spacy.load('en_core_web_sm')


08/11/2020 19:40:16 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.data.dataset_readers.dataset_reader.DatasetReader'> from params {'context_size': 3, 'token_indexers': {'token_characters': {'min_padding_length': 3, 'type': 'characters'}, 'tokens': {'lowercase_tokens': True, 'type': 'single_id'}}, 'type': 'milu'} and extras set()
08/11/2020 19:40:16 - INFO - allennlp.common.params -   dataset_reader.type = milu
08/11/2020 19:40:16 - INFO - allennlp.common.from_params -   instantiating class <class 'convlab2.nlu.milu.dataset_reader.MILUDatasetReader'> from params {'context_size': 3, 'token_indexers': {'token_characters': {'min_padding_length': 3, 'type': 'characters'}, 'tokens': {'lowercase_tokens': True, 'type': 'single_id'}}} and extras set()
08/11/2020 19:40:16 - INFO - allennlp.common.params -   dataset_reader.context_size = 3
08/11/2020 19:40:16 - INFO - allennlp.common.params -   dataset_reader.agent = None
08/11/2020 19:40:16 - INFO - allennlp.commo

아래 코드에는 user simulator가 정의되어 있습니다.

In [5]:
user_nlu = BERTNLU()
# user_nlu = MILU()
# user_nlu = SVMNLU()
user_dst = None
user_policy = RulePolicy(character='usr')
# user_policy = UserPolicyVHUS(load_from_zip=True)
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

load train, size 8434
load val, size 999
load test, size 1000
loaded train, size 113500
loaded val, size 14730
loaded test, size 14744
dialog act num: 36
sentence label num: 137
tag num: 331
intent num: 137
tag num: 331
Load from model_file param
Load from https://convlab.blob.core.windows.net/convlab-2/bert_multiwoz_all_context.zip


08/11/2020 19:40:42 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/bert_multiwoz_all_context.zip not found in cache, downloading to /tmp/tmpktbx1jy_
100%|██████████| 425713245/425713245 [05:30<00:00, 1289730.10B/s]
08/11/2020 19:46:13 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpktbx1jy_ to cache at /home/hanjy/.convlab2/cache/fe2b28201c498bb510ae89111e6bb1710a013c23920583adafc3f2b140376b90.4fbb3a3c9025fd8bc79740ec9ac9f931fb88de0cece837a569fba203fa3df2a0
08/11/2020 19:46:13 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/hanjy/.convlab2/cache/fe2b28201c498bb510ae89111e6bb1710a013c23920583adafc3f2b140376b90.4fbb3a3c9025fd8bc79740ec9ac9f931fb88de0cece837a569fba203fa3df2a0
08/11/2020 19:46:13 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpktbx1jy_


Load from /home/hanjy/samsung/200814_test/200813_dialogue/samsung_ds_practice/ConvLab-2/convlab2/nlu/jointBERT/multiwoz/output/all_context/pytorch_model.bin
bert-base-uncased


08/11/2020 19:46:18 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin not found in cache or force_download set to True, downloading to /tmp/tmpas7ga8g5
08/11/2020 19:49:10 - INFO - transformers.file_utils -   copying /tmp/tmpas7ga8g5 to cache at /home/hanjy/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
08/11/2020 19:49:10 - INFO - transformers.file_utils -   creating metadata file for /home/hanjy/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
08/11/2020 19:49:10 - INFO - transformers.file_utils -   removing temp file /tmp/tmpas7ga8g5


BERTNLU loaded
Loading goal model is done


아래 코드에는 analyzer 가 정의 되어 있습니다.

In [6]:
from convlab2.util.analysis_tool.analyzer import Analyzer

# if sys_nlu!=None, set use_nlu=True to collect more information
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=20)

dialogue:   5%|▌         | 1/20 [00:01<00:32,  1.72s/it]08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [47932720631] (slot: phone domain: taxi)
08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [57253443034] (slot: phone domain: taxi)
08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [31316650087] (slot: phone domain: taxi)
08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [85878627496] (slot: phone domain: taxi)
08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [07233561887] (slot: phone domain: taxi)
08/11/2020 19:52:47 - DEBUG - root -   Value not found in standard value set: [17343656291] (slot: phone domain: taxi)
08/11/2020 19:52:48 - DEBUG - root -   Value not found in standard value set: [65450290121] (slot: phone domain: taxi)
08/11/2020 19:52:48 - DEBUG - root -   Value not found in standard value set: [87288110405] (slot: phone domain

complete number of dialogs/tot: 0.65
success number of dialogs/tot: 0.6
average precision: 0.7517857142857143
average recall: 0.7775000000000001
average f1: 0.754532967032967
average book rate: 0.8333333333333333
average turn (succ): 14.5
average turn (all): 18.7


(0.65,
 0.6,
 0.7517857142857143,
 0.7775000000000001,
 0.754532967032967,
 0.8333333333333333,
 18.7)

## \[실습문제] Success Rate를 기준으로 상위 3가지 모델을 찾고, 아래 표를 완성하세요.

\[힌트] 여러개의 system agent를 만들고 아래의 코드를 활용해 한번에 돌리실 수 있습니다.

\[주의] 실습과제 제출시 실행기록을 반드시 남겨서 제출해야 합니다.


Rank      | NLU       | DST       | Policy    | NLG          | Success rate | Book rate | Inform P | Inform R | Inform F1 | Turn(succ/all) |
--------- | --------- | --------- | --------- | :----------: | :----------: | --------- | -------- | --------- | -------- | -------------- |
1         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
2         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
3         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |

In [7]:
set_seed(20200805)

# define your own system agent2
# sys_agent2 = PipelineAgent(...)

# define your own system agent3
# sys_agent3 = PipelineAgent(...)


# [Hint] You can append sys_agent model as much as you want!!
analyzer.compare_models(agent_list=[sys_agent, sys_agent, sys_agent], model_name=['sys_agent1', 'sys_agent2', 'sys_agent3'], total_dialog=20)

dialogue:  10%|█         | 2/20 [00:01<00:22,  1.25s/it]08/11/2020 19:53:50 - DEBUG - root -   Value not found in standard value set: [close to the area] (slot: area domain: attraction)
08/11/2020 19:53:50 - DEBUG - root -   Value not found in standard value set: [77120259724] (slot: phone domain: taxi)
dialogue:  35%|███▌      | 7/20 [00:03<00:06,  2.02it/s]08/11/2020 19:53:52 - DEBUG - root -   Value not found in standard value set: [48982244314] (slot: phone domain: taxi)
08/11/2020 19:53:52 - DEBUG - root -   Value not found in standard value set: [53867201446] (slot: phone domain: taxi)
08/11/2020 19:53:52 - DEBUG - root -   Value not found in standard value set: [98828955959] (slot: phone domain: taxi)
08/11/2020 19:53:52 - DEBUG - root -   Value not found in standard value set: [49126467648] (slot: phone domain: taxi)
08/11/2020 19:53:52 - DEBUG - root -   Value not found in standard value set: [98221217960] (slot: phone domain: taxi)
08/11/2020 19:53:52 - DEBUG - root -   Value

complete number of dialogs/tot: 0.7
success number of dialogs/tot: 0.7
average precision: 0.8412280701754387
average recall: 0.9061403508771931
average f1: 0.8346624136097819
average book rate: 0.7142857142857143
average turn (succ): 10.285714285714286
average turn (all): 18.5


dialogue:  10%|█         | 2/20 [00:01<00:16,  1.10it/s]08/11/2020 19:54:02 - DEBUG - root -   Value not found in standard value set: [close to the area] (slot: area domain: attraction)
08/11/2020 19:54:03 - DEBUG - root -   Value not found in standard value set: [77120259724] (slot: phone domain: taxi)
dialogue:  35%|███▌      | 7/20 [00:03<00:06,  1.98it/s]08/11/2020 19:54:05 - DEBUG - root -   Value not found in standard value set: [48982244314] (slot: phone domain: taxi)
08/11/2020 19:54:05 - DEBUG - root -   Value not found in standard value set: [53867201446] (slot: phone domain: taxi)
08/11/2020 19:54:05 - DEBUG - root -   Value not found in standard value set: [98828955959] (slot: phone domain: taxi)
08/11/2020 19:54:05 - DEBUG - root -   Value not found in standard value set: [49126467648] (slot: phone domain: taxi)
08/11/2020 19:54:05 - DEBUG - root -   Value not found in standard value set: [98221217960] (slot: phone domain: taxi)
08/11/2020 19:54:05 - DEBUG - root -   Value

complete number of dialogs/tot: 0.7
success number of dialogs/tot: 0.7
average precision: 0.8412280701754387
average recall: 0.9061403508771931
average f1: 0.8346624136097819
average book rate: 0.7142857142857143
average turn (succ): 10.285714285714286
average turn (all): 18.5


dialogue:  10%|█         | 2/20 [00:01<00:15,  1.16it/s]08/11/2020 19:54:14 - DEBUG - root -   Value not found in standard value set: [close to the area] (slot: area domain: attraction)
08/11/2020 19:54:15 - DEBUG - root -   Value not found in standard value set: [77120259724] (slot: phone domain: taxi)
dialogue:  35%|███▌      | 7/20 [00:03<00:05,  2.17it/s]08/11/2020 19:54:17 - DEBUG - root -   Value not found in standard value set: [48982244314] (slot: phone domain: taxi)
08/11/2020 19:54:17 - DEBUG - root -   Value not found in standard value set: [53867201446] (slot: phone domain: taxi)
08/11/2020 19:54:17 - DEBUG - root -   Value not found in standard value set: [98828955959] (slot: phone domain: taxi)
08/11/2020 19:54:17 - DEBUG - root -   Value not found in standard value set: [49126467648] (slot: phone domain: taxi)
08/11/2020 19:54:17 - DEBUG - root -   Value not found in standard value set: [98221217960] (slot: phone domain: taxi)
08/11/2020 19:54:17 - DEBUG - root -   Value

complete number of dialogs/tot: 0.7
success number of dialogs/tot: 0.7
average precision: 0.8412280701754387
average recall: 0.9061403508771931
average f1: 0.8346624136097819
average book rate: 0.7142857142857143
average turn (succ): 10.285714285714286
average turn (all): 18.5
